In [ ]:
# Import libraries for OS interaction, data handling, NLP modeling, logging, and randomness
import os
import torch
import pandas as pd
from typing import Dict, Optional, List
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import logging
import random

# Configure and initialize a logger to track the program's execution
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Load pretrained tokenizer and model (FLAN-T5) for sequence-to-sequence tasks
model_name = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForSeq2SeqLM.from_pretrained(
    model_name,
    device_map="cpu",
    torch_dtype=torch.float32 
)

# Define sample customer emails with metadata for testing the model
sample_emails = [
    {
        "id": "001",
        "from": "angry.customer@example.com",
        "subject": "Broken product received",
        "body": "I received my order #12345 yesterday but it arrived completely damaged. This is unacceptable and I demand a refund immediately. This is the worst customer service I've experienced.",
        "timestamp": "2024-03-15T10:30:00Z"
    },
    {
        "id": "002",
        "from": "curious.shopper@example.com",
        "subject": "Question about product specifications",
        "body": "Hi, I'm interested in buying your premium package but I couldn't find information about whether it's compatible with Mac OS. Could you please clarify this? Thanks!",
        "timestamp": "2024-03-15T11:45:00Z"
    },
    {
        "id": "003",
        "from": "happy.user@example.com",
        "subject": "Amazing customer support",
        "body": "I just wanted to say thank you for the excellent support I received from Sarah on your team. She went above and beyond to help resolve my issue. Keep up the great work!",
        "timestamp": "2024-03-15T13:15:00Z"
    },
    {
        "id": "004",
        "from": "tech.user@example.com",
        "subject": "Need help with installation",
        "body": "I've been trying to install the software for the past hour but keep getting error code 5123. I've already tried restarting my computer and clearing the cache. Please help!",
        "timestamp": "2024-03-15T14:20:00Z"
    },
    {
        "id": "005",
        "from": "business.client@example.com",
        "subject": "Partnership opportunity",
        "body": "Our company is interested in exploring potential partnership opportunities with your organization. Would it be possible to schedule a call next week to discuss this further?",
        "timestamp": "2024-03-15T15:00:00Z"
    },
    {
        "id": "006",
        "from": "frustrated.buyer@example.com",
        "subject": "Missing items in my order",
        "body": "My order #67890 arrived today, but two items are missing. I paid for express shipping and this is very disappointing. Please resolve this immediately.",
        "timestamp": "2024-03-15T16:10:00Z"
    },
    {
        "id": "007",
        "from": "potential.partner@example.com",
        "subject": "Collaboration Inquiry",
        "body": "Hello, I represent a startup that aligns closely with your values and offerings. We’d love to explore ways we can collaborate in the near future.",
        "timestamp": "2024-03-15T17:00:00Z"
    },
    {
        "id": "008",
        "from": "feedback.sender@example.com",
        "subject": "Suggestion for improvement",
        "body": "Your app is quite useful, but it would be great if it allowed offline access. Many users, including myself, travel frequently and would benefit from this feature.",
        "timestamp": "2024-03-15T17:45:00Z"
    },
    {
        "id": "009",
        "from": "confused.client@example.com",
        "subject": "Billing discrepancy",
        "body": "I was charged twice for the same service on my credit card ending in 8812. Could you please look into this and refund the duplicate charge?",
        "timestamp": "2024-03-15T18:30:00Z"
    },
    {
        "id": "010",
        "from": "grateful.customer@example.com",
        "subject": "Thanks for the quick response",
        "body": "I really appreciate the speed with which your team resolved my login issue. It’s refreshing to see such efficiency. Keep it up!",
        "timestamp": "2024-03-15T19:15:00Z"
    },
    {
        "id": "011",
        "from": "event.organizer@example.com",
        "subject": "Request for sponsorship",
        "body": "We're organizing a tech event in San Francisco this June and would be honored to have your brand as a sponsor. Let us know if you’re interested.",
        "timestamp": "2024-03-15T20:05:00Z"
    },
    {
        "id": "012",
        "from": "loyal.client@example.com",
        "subject": "Inquiry about loyalty program",
        "body": "I’ve been a customer for over 3 years and was wondering if you have a loyalty or rewards program. I’d love to take part if so.",
        "timestamp": "2024-03-15T20:45:00Z"
    },
    {
        "id": "013",
        "from": "non.working.code@example.com",
        "subject": "Discount code not working",
        "body": "I received a promo code (WELCOME10) in your newsletter, but it shows as invalid at checkout. Could you please check what's wrong?",
        "timestamp": "2024-03-15T21:20:00Z"
    },
    {
        "id": "014",
        "from": "early.adopter@example.com",
        "subject": "Beta access request",
        "body": "I saw your announcement about the new feature release and would love to be part of the beta testing group. Let me know if that’s possible.",
        "timestamp": "2024-03-15T21:55:00Z"
    },
    {
        "id": "015",
        "from": "support.seeker@example.com",
        "subject": "Problem logging into account",
        "body": "Every time I try to log into my account, I get a message saying the credentials are invalid—even after resetting my password. Please assist.",
        "timestamp": "2024-03-15T22:30:00Z"
    },
    {
        "id": "016",
        "from": "shipping.issue@example.com",
        "subject": "Package delayed",
        "body": "My order #45678 was supposed to arrive two days ago but the tracking info hasn’t updated. Can you provide an update or issue a refund?",
        "timestamp": "2024-03-15T23:05:00Z"
    },
    {
        "id": "017",
        "from": "feature.requester@example.com",
        "subject": "Request for dark mode",
        "body": "Could you consider adding a dark mode to your application? It would make using it at night much more comfortable.",
        "timestamp": "2024-03-15T23:45:00Z"
    },
    {
        "id": "018",
        "from": "account.manager@example.com",
        "subject": "Corporate account inquiry",
        "body": "I manage procurement for a mid-sized firm and we’re exploring options for a corporate account. Please send more information on pricing and terms.",
        "timestamp": "2024-03-16T00:15:00Z"
    },
    {
        "id": "019",
        "from": "user.testimonial@example.com",
        "subject": "Positive experience with your app",
        "body": "I’ve been using your mobile app for a month now and it’s genuinely improved my daily routine. Just wanted to share my appreciation!",
        "timestamp": "2024-03-16T00:50:00Z"
    },
    {
        "id": "020",
        "from": "cancel.request@example.com",
        "subject": "Subscription cancellation",
        "body": "Please cancel my subscription effective immediately. I’ve already found an alternative and no longer wish to be billed.",
        "timestamp": "2024-03-16T01:20:00Z"
    }
]


# Generates personalized, empathetic responses to emails based on their classification, ensuring clarity and
# relevance. Uses a language model to identify the email category and generate a direct, appropriate response.
#Prioritizes action and accuracy in each reply.
class EmailProcessor:
    # Initialize valid email categories for classification
    def __init__(self):
        self.valid_categories = {
            "complaint", "inquiry", "feedback", "support_request", "other",
            "praise", "partnership_opportunity", "feature_request", "billing_issue",
            "technical_issue", "cancellation_request", "product_question", "legal_complaint",
            "job_application", "spam"
        }
    def classify_email(self, email: Dict) -> Optional[str]:
        """
        Classifies an email based on its content into one of the predefined categories.

        This method reads the content of the email and determines its main intent, classifying it into categories such as "complaint", "inquiry", "support_request", etc. The classification is determined by analyzing the text and matching it to the most appropriate category. If the content doesn't fit any of the predefined categories, it is classified as "other".

        :param email: A dictionary containing the email's details, including its "body".
        :return: A string representing the category of the email (e.g., "complaint", "inquiry", etc.), or "other" if the classification fails.

        Example:
        >>> email = {"body": "I would like to know the status of my order."}
        >>> classification = processor.classify_email(email)
        >>> print(classification)  # "inquiry"

        The method uses a prompt to instruct an AI model to categorize the email based on the user's intent, and it returns the category name.
        """
        email_body = email["body"]
        prompt = f"""
        You are a customer service AI trained to classify email content based on its main purpose. Your job is to read the email below and assign it to ONE AND ONLY ONE category from the following list:

        - complaint (negative feedback expressing dissatisfaction)
        - inquiry (asking for information or clarification)
        - feedback (neutral or mixed user experience or suggestions)
        - support_request (asking for technical help, update, or assistance)
        - other (does not clearly fit any category)
        - praise (positive feedback or compliments)
        - partnership_opportunity (business proposal or collaboration interest)
        - feature_request (suggestion for new features or services)
        - billing_issue (problems related to payments or charges)
        - technical_issue (problems with system/software/hardware function)
        - cancellation_request (wants to cancel service/order)
        - product_question (asking about specs, availability, compatibility)
        - legal_complaint (privacy, terms, data handling concerns)
        - job_application (career-related communication)
        - spam (irrelevant or mass promotional content)

        **Choose the category that best represents the user's main intent. Prioritize the action the user is expecting.**

        Respond ONLY with the category name (no explanation, no punctuation):

        Email:
        \"\"\"{email_body}\"\"\"

        Category:
        """
        try:
            inputs = tokenizer(prompt, return_tensors="pt", padding=True)
            outputs = model.generate(
                **inputs,
                max_length=30,
                temperature=0.7,
                top_p=0.9,
                repetition_penalty=1.2,
                pad_token_id=tokenizer.eos_token_id
            )
            result = tokenizer.decode(outputs[0], skip_special_tokens=True)
            category = result.strip().lower()
            for cat in self.valid_categories:
                if cat in category:
                    return cat
        except Exception as e:
            logger.error(f"Classification failed: {str(e)}")
        return "other"
    
    def generate_response(self, email: Dict, classification: str) -> Optional[str]:
        """
        Generates a professional and empathetic response to an email based on its classification.

        This method takes the content of the email and its classification (such as "complaint", "inquiry", 
        "support_request", etc.), and generates a concise, empathetic response tailored to the situation. 
        The response is then improved to ensure it is polite, clear, and actionable.

        :param email: A dictionary containing the email's details, including its "body" and "id".
        :param classification: A string indicating the classification of the email, which determines the type of response.
        :return: A refined, professional response based on the email's classification, or None if an error occurs during processing.
        
        Example:
        >>> email = {"body": "I am experiencing issues with my order.", "id": "12345"}
        >>> classification = "complaint"
        >>> response = processor.generate_response(email, classification)
        >>> print(response)  # A concise, empathetic response for the complaint.

        The method uses a predefined template based on the classification and refines the response by:
        - Removing repetitive or vague phrases.
        - Adding specific solutions where needed.
        - Ensuring the response is clear and action-oriented.
        - Using a tone that is professional, neutral, and concise.
        """
        email_body = email["body"]
        email_id = email["id"]

        template = {
            "complaint": f"Hello, we sincerely apologize for the inconvenience you've experienced. Your case has been logged under ID {email_id}, and our team is actively working to resolve it within the next 24 hours. We will keep you updated on the progress.",
            "inquiry": "Thank you for your inquiry. Our team is reviewing your question, and one of our specialists will reach out soon with the information you need.",
            "feedback": "Thank you for sharing your feedback with us! We greatly value your suggestions, and they help us continuously improve our services.",
            "support_request": f"Your request has been received and assigned to Ticket ID: support-{email_id}. A technician will contact you shortly to assist you.",
            "other": "We’ve received your message and will carefully review it to provide you with an appropriate response as soon as possible.",
            "praise": "We’re thrilled to hear that you’ve had a positive experience! Your words motivate our entire team. Thank you for sharing them with us!",
            "partnership_opportunity": "We’re excited about your interest in collaborating with us. Our business development team will review your proposal and reach out soon.",
            "feature_request": "We appreciate your suggestion. It has been shared with our product team for consideration in future updates.",
            "billing_issue": f"We understand your concern regarding billing. Your case (ID: {email_id}) is being reviewed, and our billing team will reach out shortly.",
            "technical_issue": f"We are investigating the technical issue you reported (Support ID: tech-{email_id}). A specialist will contact you as soon as possible.",
            "cancellation_request": f"We’ve received your cancellation request (ref: {email_id}) and it’s being processed. We will confirm the status shortly.",
            "product_question": "We are reviewing your product question and will provide the requested details very soon.",
            "legal_complaint": f"Your legal concern has been registered (Case ID: {email_id}) and is being escalated to our legal team for review.",
            "job_application": "Thank you for your interest in joining our team. We’ve received your application and will contact you if there is a suitable match with our current openings.",
            "spam": "This message has been flagged as spam and will not receive a response. If you believe this is an error, please reach out to our support team."
        }.get(classification, "Thank you for reaching out to us. We’ll be happy to assist you shortly.")

        prompt = f"""
        You are a professional customer support agent. Based on the email content and its classification as `{classification}`, review the following response and rewrite it to ensure it is empathetic, polite, and concise (maximum 2 sentences).

        Your task includes:
        - If the response contains repeated phrases or mirrors the original email unnecessarily, remove or rewrite them.
        - If the user's message reflects a problem, include a specific and immediate solution in your reply.
        - If the response is vague or generic, make it more informative and action-oriented.
        - If the user's message includes a question, ensure it is clearly and directly answered.
        - If the classification is `support_request`, always provide a short and concrete solution.

        Guidelines:
        - Do not repeat the user's original message.
        - Do not include greetings or sign-offs.
        - Respond directly to the point with clarity.
        - Avoid empty or generic phrases.
        - Maintain a professional and neutral tone.
        - If the information provided is ambiguous, specify what is missing.
        - Do not repeat words or phrases unnecessarily.
        - Always prioritize clarity and actionable support.

        Email: \"\"\"{email_body}\"\"\"
        Category: {classification}
        Initial response: \"\"\"{template}\"\"\"

        Improved response:
        """
        try:
            inputs = tokenizer(prompt, return_tensors="pt", padding=True)
            outputs = model.generate(
                **inputs,
                max_length=50,
                do_sample=True, 
                temperature=0.7,
                top_p=0.9,
                top_k=50,
                repetition_penalty=1.2,
                no_repeat_ngram_size=3,
                length_penalty=1.0,
                early_stopping=False,
                num_beams=5, 
                pad_token_id=tokenizer.eos_token_id,
            )
            result = tokenizer.decode(outputs[0], skip_special_tokens=True)
            return result.strip().split("Improved response:")[-1].strip()
        except Exception as e:
            logger.error(f"Response generation failed: {str(e)}")
            return None

class EmailAutomationSystem:
    def __init__(self, processor: EmailProcessor):
        """
        Initializes the EmailAutomationSystem with an EmailProcessor instance.

        This constructor accepts an EmailProcessor instance, which is used for classifying emails and 
        generating responses. The processor is stored as a class attribute for later use in email processing.

        :param processor: An instance of the EmailProcessor class responsible for classifying and generating responses.
        
        Example:
        >>> processor = EmailProcessor()
        >>> system = EmailAutomationSystem(processor)
        >>> print(system.processor)  # Should show the processor instance.
        """
        self.processor = processor
        
    def process_email(self, email: Dict) -> Dict:
        """
        Processes an email by classifying it and generating an appropriate response.

        This function takes an email, classifies it based on its content using the EmailProcessor, 
        and generates a personalized response. If any error occurs during processing, it handles 
        the exception and returns a dictionary with the classification and response as None.

        :param email: A dictionary containing the email's information, including an 'id' and 'body'.
        :return: A dictionary with the email's id, its classification, and the generated response.

        :raises Exception: If an error occurs during classification or response generation, it is caught 
                            and handled, returning None for classification and response.

        Example:
        >>> email = {'id': 1, 'body': 'I have a problem with my billing.'}
        >>> processor = EmailProcessor()
        >>> system = EmailAutomationSystem(processor)
        >>> result = system.process_email(email)
        >>> print(result)
        {'email_id': 1, 'classification': 'billing_issue', 'response': 'We understand your concern regarding billing...'}
        """
        try:
            classification = self.processor.classify_email(email)
            response = self.processor.generate_response(email, classification)

            return {
                "email_id": email["id"],
                "classification": classification,
                "response": response
            }
        except Exception as e:
            # Error handling if there's an issue with processing
            print(f"Error processing email {email['id']}: {str(e)}")
            return {
                "email_id": email["id"],
                "classification": None,
                "response": None
            }

def get_random_emails(source_list: List[Dict], num_samples: int = 5) -> List[Dict]:
    """
    Retrieves a random sample of emails from the source list.

    This function selects a specified number of random emails from a provided list of emails.

    :param source_list: A list of dictionaries containing email data.
    :param num_samples: The number of random samples to retrieve (default is 5).
    :return: A list of dictionaries containing the randomly selected emails.

    :raises ValueError: If the source list has fewer elements than the specified num_samples.

    Example:
    >>> sample_emails = [{'id': 1, 'body': '...'}, {'id': 2, 'body': '...'}, ...]
    >>> random_emails = get_random_emails(sample_emails, 3)
    >>> print(random_emails)
    """
    if len(source_list) < num_samples:
        raise ValueError(f"La lista tiene menos de {num_samples} elementos. No se pueden obtener tantas muestras.")
    
    random_emails = random.sample(source_list, num_samples)
    return random_emails

def run_demonstration():
    """
    Runs a demonstration of the email processing system.

    This function demonstrates the email classification and response generation process by:
    1. Initializing the EmailProcessor and EmailAutomationSystem objects.
    2. Selecting a random sample of emails from a provided list.
    3. Processing the selected emails by classifying them and generating appropriate responses.
    4. Storing the results in a DataFrame for easy visualization.
    5. Printing the selected sample emails and their processed results (classification and response) in a structured format.

    The demonstration includes:
    - Displaying the selected emails.
    - Providing a summary of classifications and generated responses in a DataFrame.

    Expected Output:
    - A list of sample emails and their classifications.
    - A summary table with email ID, classification, and response.
    """
    processor = EmailProcessor()
    system = EmailAutomationSystem(processor)

    selected_emails = get_random_emails(sample_emails)
    results = [system.process_email(email) for email in selected_emails]
    df = pd.DataFrame(results)

    print("\n=== Sample Emails ===")
    for email in selected_emails:
        print(email)
        print("-" * 80)

    print("\n=== Processing Summary ===")
    print(df)

if __name__ == "__main__":
    pd.set_option("display.max_colwidth", None)
    run_demonstration()

c:\Users\santy\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\santy\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\generation\configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
c:\Users\santy\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\generation\configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(



=== Sample Emails ===
{'id': '018', 'from': 'account.manager@example.com', 'subject': 'Corporate account inquiry', 'body': 'I manage procurement for a mid-sized firm and we’re exploring options for a corporate account. Please send more information on pricing and terms.', 'timestamp': '2024-03-16T00:15:00Z'}
--------------------------------------------------------------------------------
{'id': '017', 'from': 'feature.requester@example.com', 'subject': 'Request for dark mode', 'body': 'Could you consider adding a dark mode to your application? It would make using it at night much more comfortable.', 'timestamp': '2024-03-15T23:45:00Z'}
--------------------------------------------------------------------------------
{'id': '009', 'from': 'confused.client@example.com', 'subject': 'Billing discrepancy', 'body': 'I was charged twice for the same service on my credit card ending in 8812. Could you please look into this and refund the duplicate charge?', 'timestamp': '2024-03-15T18:30:00Z'}
